Had to pip install

*   !sudo pip install awscli
*   [Create access key](https://www.geeksforgeeks.org/how-to-create-aws_access_key-and-seceret-key/)
*   !aws configure
*   !aws s3 sync s3://open-logo-s3/branded/ .



In [11]:
!aws s3 sync s3://open-logo-s3/branded/ .

download: s3://open-logo-s3/branded/andes/im.jpg6250.jpg to andes/im.jpg6250.jpg      
download: s3://open-logo-s3/branded/arbys/im.jpg7638.jpg to arbys/im.jpg7638.jpg        
download: s3://open-logo-s3/branded/big_gulp/im.jpg6419.jpg to big_gulp/im.jpg6419.jpg  
download: s3://open-logo-s3/branded/arbys/im.jpg245.jpg to arbys/im.jpg245.jpg          
download: s3://open-logo-s3/branded/bud_light/im.jpg1307.jpg to bud_light/im.jpg1307.jpg
download: s3://open-logo-s3/branded/big_gulp/im.jpg9376.jpg to big_gulp/im.jpg9376.jpg  
download: s3://open-logo-s3/branded/bud_light/im.jpg1312.jpg to bud_light/im.jpg1312.jpg
download: s3://open-logo-s3/branded/bud_light/im.jpg102.jpg to bud_light/im.jpg102.jpg  
download: s3://open-logo-s3/branded/andes/im.jpg6251.jpg to andes/im.jpg6251.jpg        
download: s3://open-logo-s3/branded/big_gulp/im.jpg6467.jpg to big_gulp/im.jpg6467.jpg  
download: s3://open-logo-s3/branded/bud_light/im.jpg1163.jpg to bud_light/im.jpg1163.jpg
download: s3://open-log

Get the embeddings of all the images from each class

In [13]:
# Import all libraries
import torch
import torch.nn.functional as F
from PIL import Image
from transformers import BeitFeatureExtractor, BeitModel
from tqdm import tqdm
import os


In [28]:
for class_dir in os.listdir(data_dir):
  print(os.path.realpath(class_dir))

/content/.config
/content/canada_dry
/content/guinness
/content/dasani
/content/modelo
/content/goldfish
/content/natural_light
/content/nestle_pure_life
/content/ritz
/content/monster
/content/white_claw
/content/lays
/content/poland_spring
/content/jarritos
/content/dunkin
/content/great_value
/content/bud_light
/content/rice_krispies 
/content/milwaukees_best
/content/marlboro
/content/mcdonalds
/content/heineken
/content/quick_trip
/content/arbys
/content/green_mountain
/content/blue_moon
/content/stella
/content/.ipynb_checkpoints
/content/newport
/content/nabisco
/content/goya
/content/big_gulp
/content/hersheys
/content/kirkland
/content/burger_king
/content/corona
/content/seven_eleven
/content/busch_light
/content/coors
/content/lifestyles
/content/twizzlers
/content/target
/content/senorial
/content/mikes_hard
/content/coca_cola
/content/smirnoff
/content/mountain_dew
/content/fireball
/content/dr_pepper
/content/deer_park
/content/miller_light
/content/andes
/content/wawa
/c

In [41]:
import numpy as np

data_dir = '.'

# Initialize lists to hold the images and labels
images = []
labels = []

# Loop through each class directory
for class_dir in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_dir)

    # Loop through each image in the class directory
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)

        # include "if" statement to not include .ipynb and
        if img_path.endswith('.jpg'):

          # Load image
          #image = Image.open(img_path)

          #Add image to the list
          images.append(img_path)

          # create labels using the name of the directory
          labels.append(class_dir)

# Convert lists to numpy arrays
#images = np.array(images)
#labels = np.array(labels)

print(len(images))
print(len(labels))


2448
2448


In [51]:
labels[4]

'guinness'

In [52]:
def load_image(image_path):
    return Image.open(image_path).convert("RGB")

def generate_beit_embeddings(image_paths, processor, model):
    '''
    This function will generate BEiT Image Embeddings of the logo dataset.
    Output is (n, 768) where n represents the number of images in the logo dataset.

    image_paths: list of all image paths
    processor: image processor in the form of BeitFeatureExtractor.from_pretrained('microsoft/beit-base-patch16-224')
    model: a model in the form of BeitModel.from_pretrained('microsoft/beit-base-patch16-224')
    '''
    embeddings = []

    for image_path in tqdm(image_paths):
        image = load_image(image_path)

        # use inputted processor to preprocess the image
        inputs = processor(images=image, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)

            # get the last hidden state
            last_hidden_states = outputs.last_hidden_state

            # Apply mean pooling to the hidden state
            pooled_embedding = torch.mean(last_hidden_states, dim=1)

            # Add outputted embedding to the list of embeddings
            embeddings.append(pooled_embedding)#pooled_embedding #embeddings.append(pooled_embedding)

    return embeddings#torch.cat(embeddings, dim=0)


def calculate_cosine_similarity_matrix(X, Y):
    '''
    This function calculates cosine similarity between X and Y inputs.
    '''
    # Normalize the embeddings
    X_normalized = F.normalize(X, p=2, dim=1)
    Y_normalized = F.normalize(Y, p=2, dim=1)

    # Calculate cosine similarity
    cosine_similarity_matrix = torch.mm(X_normalized, Y_normalized.t())
    return cosine_similarity_matrix


def calculate_cosine_similarity_matrix2(X, Y):
  from numpy.linalg import norm
  all_similarities = []


  for i, x_embedding in enumerate(X):
      similarities = []
      for y_embedding in Y:
        cosine = ((x_embedding * y_embedding).sum(dim=-1) / (torch.norm(x_embedding, dim=-1) * torch.norm(y_embedding, dim=-1))).mean().item()

        similarities.append(cosine)
      all_similarities.append(similarities)
  return (all_similarities)




feature_extractor = BeitFeatureExtractor.from_pretrained('microsoft/beit-base-patch16-224')
model = BeitModel.from_pretrained('microsoft/beit-base-patch16-224')


#dataset_dir = '.'
#labels = ("andes", "arbys")
#image_paths = [os.path.join(dataset_dir, fname, each_image) for fname in os.listdir(dataset_dir) for each_image in os.listdir(dataset_dir+"/"+fname) if fname.endswith(labels) if each_image.endswith(('.png', '.jpg', '.jpeg'))]
#print(image_paths)
X = generate_beit_embeddings(images, feature_extractor, model)

#len(X)

/usr/local/lib/python3.10/dist-packages/transformers/models/beit/feature_extraction_beit.py:28: FutureWarning: The class BeitFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use BeitImageProcessor instead.
  warnings.warn(
100%|██████████| 2448/2448 [36:55<00:00,  1.10it/s]


In [59]:
print(len(X))
X[0].shape

2448


torch.Size([1, 768])

In [ ]:
# X_tensor.shape will be torch.Size([n, 768]) where n is the number of images
# use dim = 0 to concatenate along the batch dimension
X_tensor = torch.cat(X, dim=0)

# Convert the tensor to a numpy array. Output will be (n, 768)
X_array = X_tensor.numpy()

Input image

In [128]:
dataset_dir = [x for x in os.listdir() if x.endswith(".jpg")]

input_img = generate_beit_embeddings(dataset_dir, feature_extractor, model)

input_img_array = torch.cat(input_img, dim = 0).numpy()



image path afe74453-8e45-44ee-96f3-d53405dae9f6.jpg


Work is based off this [source](https://www.datacamp.com/tutorial/svm-classification-scikit-learn-python)

In [129]:
from sklearn.preprocessing import LabelEncoder
from sklearn import svm

le = LabelEncoder()


#Fit label encoder and return encoded labels.
encoded_labels = le.fit_transform(labels)

#Create a svm Classifier
clf = svm.SVC()

clf.fit(X_array, encoded_labels)


SVC()

In [130]:
y_pred = clf.predict(input_img_array)

In [131]:
le.inverse_transform(y_pred)

array(['gatorade'], dtype='<U16')

Encode the labels using `LabelEncoder`